# **Data Analysis**

In [1]:
from data_analysis import *

oecd_df = pd.read_csv("../04 Data/OECD/OECD_integrated.csv")
oecd_df.set_index("TIME_PERIOD", verify_integrity=True, inplace=True, drop=True)
oecd_df.drop(columns=["Unnamed: 0"], inplace=True)
oecd_df.index = pd.to_datetime(oecd_df.index)
oecd_df.info()

sf_df = pd.read_csv("../04 Data/Salesforce/Salesforce_OpportunityHistory_Timeseries.csv")
sf_df.set_index("TIME_PERIOD", verify_integrity=True, inplace=True, drop=True)
sf_df.index = pd.to_datetime(sf_df.index)
sf_df.info()

sf_cleaned_df = pd.read_csv("../04 Data/Salesforce/Salesforce_OpportunityHistory_Timeseries_cleaned.csv")
sf_cleaned_df.set_index("TIME_PERIOD", verify_integrity=True, inplace=True, drop=True)
sf_cleaned_df.drop(columns=["index"], inplace=True)
sf_cleaned_df.index = pd.to_datetime(sf_cleaned_df.index)
sf_cleaned_df.info()

stocks_df = pd.read_csv("../04 Data/Stock Movements Yahoo Finance/Stocks_integrated.csv")
stocks_df.rename(columns={"Date": "TIME_PERIOD"}, inplace=True)
stocks_df.set_index("TIME_PERIOD", verify_integrity=True, inplace=True, drop=True)
stocks_df.index = pd.to_datetime(stocks_df.index)
stocks_df.info()

gt_df = pd.read_csv("../04 Data/Google Trends/GoogleTrends_integrated.csv")
gt_df.set_index("TIME_PERIOD", verify_integrity=True, inplace=True, drop=True)
gt_df.index = pd.to_datetime(gt_df.index)
gt_df.info()


hg_oi_df = pd.read_csv("../04 Data/HOMAG/OrderIntake_series_monthly.csv")
hg_oi_df.set_index("TIME_PERIOD", verify_integrity=True, inplace=True, drop=True)
hg_oi_df.index = pd.to_datetime(hg_oi_df.index)
hg_oi_df.info()

hg_oi_10y_df = pd.read_csv("../04 Data/HOMAG/OrderIntake_series_monthly_10y.csv")
hg_oi_10y_df.set_index("TIME_PERIOD", verify_integrity=True, inplace=True, drop=True)
hg_oi_10y_df.index = pd.to_datetime(hg_oi_10y_df.index)
hg_oi_10y_df.info()

hg_oi_10y_df_11_24 = pd.read_csv("../04 Data/HOMAG/2024_12_16_OrderIntake_series_monthly_10y.csv")
hg_oi_10y_df_11_24.set_index("TIME_PERIOD", verify_integrity=True, inplace=True, drop=True)
hg_oi_10y_df_11_24.index = pd.to_datetime(hg_oi_10y_df_11_24.index)
hg_oi_10y_df_11_24.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1272 entries, 1919-01-01 to 2024-12-01
Columns: 171 entries, OECD_MEI_FIN_IRLT_USA to OECD_MATERIAL_RESOURCES_MF_OECDE
dtypes: float64(171)
memory usage: 1.7 MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 198 entries, 2024-11-01 to 2008-06-01
Columns: 226 entries, Avg Stage Duration Open to New Stage Qualify Expected Amount
dtypes: float64(182), int64(44)
memory usage: 351.1 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 198 entries, 2024-11-01 to 2008-06-01
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   New Amount            198 non-null    float64
 1   Open Expected Amount  198 non-null    float64
 2   Closed Amount         198 non-null    float64
 3   Won Amount            198 non-null    float64
 4   Open Amount           198 non-null    float64
 5   Won Count             198 non-null    int64  
dtypes: float64(5), int64(

Auftragseingang nach Monat

In [ ]:
def plot_monthly_boxplot(df, column_name, title_part):
    df.index = pd.to_datetime(df.index)
    oldest_date = df.index.min()
    month_year = oldest_date.strftime("%B %Y")
    df['Monat'] = df.index.month
    
    plt.figure(figsize=(12, 6))
    df.boxplot(column=column_name, by='Monat', grid=False, figsize=(11,6))
    
    plt.title(f'Boxplot {title_part} pro Monat seit {month_year}')
    plt.suptitle('')
    plt.xlabel('Monat')
    plt.ylabel('Wert in €')
    plt.xticks(ticks=range(1, 13), labels=['Januar', 'Februar', 'März', 'April', 'Mai', 'Juni', 
                                           'Juli', 'August', 'September', 'Oktober', 'November', 'Dezember'])
    plt.show()

oi_box_plot_df = hg_oi_10y_df_11_24[hg_oi_10y_df_11_24.index >= pd.to_datetime("2016-01")].copy()
plot_monthly_boxplot(oi_box_plot_df, 'T', 'Auftragseingang')

Auftragseingang je BU über die Zeit

In [ ]:
highlight_columns = ['T']
oi_plot_df = hg_oi_10y_df_11_24.copy()
oi_plot_df.index.rename("Date", inplace=True)
oi_plot_df.reset_index(drop=False, inplace=True)
plot, plot_imgstream = create_plot(
    df=oi_plot_df, 
    highlight_columns=highlight_columns, 
    color_palette=color_palette,
    chart_title='Order Intake',
    xlabel='Datum',
    ylabel='Wert (in M€)',
    scale_factor=1e-6
)
plot.show()

## Analyzing Relationships between Order Intake and Indicators for Correlation, Covariance and Precedence

In [6]:
# predictors_df = oecd_df
# predictors_df = sf_df
# predictors_df = stocks_df
# predictors_df = gt_df
# predictors_df = sf_cleaned_df
predictors_df = pd.concat([oecd_df, sf_cleaned_df, stocks_df, gt_df], axis=1)

results = compute_relationship_analysis(target_df=hg_oi_10y_df_11_24[["T"]], predictors_df=predictors_df)

Länge: 203, Lag: 12

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.9615  , p=0.3280  , df_denom=199, df_num=1
ssr based chi2 test:   chi2=0.9760  , p=0.3232  , df=1
likelihood ratio test: chi2=0.9737  , p=0.3238  , df=1
parameter F test:         F=0.9615  , p=0.3280  , df_denom=199, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.7879  , p=0.1700  , df_denom=196, df_num=2
ssr based chi2 test:   chi2=3.6670  , p=0.1599  , df=2
likelihood ratio test: chi2=3.6340  , p=0.1625  , df=2
parameter F test:         F=1.7879  , p=0.1700  , df_denom=196, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.3940  , p=0.2459  , df_denom=193, df_num=3
ssr based chi2 test:   chi2=4.3337  , p=0.2276  , df=3
likelihood ratio test: chi2=4.2875  , p=0.2320  , df=3
parameter F test:         F=1.3940  , p=0.2459  , df_denom=193, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=

In [21]:
n_best = 50
best_predictors = results.reindex(results['Best Granger Causality P-Value'].sort_values(ascending=True).index).head(n_best)
best_correlations = results.reindex(results['Best Correlation'].abs().sort_values(ascending=False).index).head(n_best)
best_covariants = results.reindex(results['Best Covariance'].abs().sort_values(ascending=False).index).head(n_best)

def get_best_predictors_df_by_th(analysis_results_df, threshold=0.05):
    return analysis_results_df.reindex(analysis_results_df['Best Granger Causality P-Value'].sort_values(ascending=True).index)[analysis_results_df['Best Granger Causality P-Value'] < threshold][["Predictor", "Best Granger Causality P-Value"]]
def get_best_predictors_by_th(analysis_results_df, threshold=0.05):
    return get_best_predictors_df_by_th(analysis_results_df, threshold)["Predictor"].tolist()

best_predictors_by_th = get_best_predictors_df_by_th(results, 0.1)
best_predictors_by_th_names = get_best_predictors_by_th(results, 0.1)
best_predictors_by_th

C:\Users\tb10muj\AppData\Local\Temp\ipykernel_4560\2818300702.py:7: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\tb10muj\AppData\Local\Temp\ipykernel_4560\2818300702.py:7: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,Predictor,Best Granger Causality P-Value
246,GT Internet der Dinge,9.369810e-07
127,OECD_PRICES_CPI_HWE_SWE,1.408602e-06
55,OECD_MEI_REAL_PRI_CHE,1.570413e-05
172,Open Expected Amount,1.982234e-05
116,OECD_PRICES_CPI_FHE_AUT,2.471833e-05
...,...,...
11,OECD_MEI_FIN_IRST_CHN,8.693075e-02
174,Won Amount,8.774888e-02
24,OECD_MEI_FIN_IRII_DNK,8.793998e-02
206,SPI_High,9.195682e-02


In [ ]:
plot_series_with_predictors(target_df=hg_oi_10y_df_11_24, predictors_df=predictors_df, best_predictors_df=best_predictors, n_best=15, title="Best Predictors for Order Intake")
display(best_predictors[["Target", "Predictor", "Best Granger Causality P-Value", "Best Granger Causality P-Value Lag", "Granger Causality P-Values"]].head(50))

Precedence (Granger Causality)

In [ ]:
plot_series_with_predictors(target_df=hg_oi_10y_df, predictors_df=oecd_df, best_predictors_df=best_predictors, n_best=10, title="Best Predictors for Order Intake")
best_predictors[["Target", "Predictor", "Best Granger Causality P-Value", "Best Granger Causality P-Value Lag", "Granger Causality P-Values"]]

Correlation

In [ ]:
plot_series_with_predictors(target_df=hg_oi_10y_df, predictors_df=oecd_df, best_predictors_df=best_correlations, n_best=10, title="Strongest Correlating Indicators for Order Intake")
best_correlations[["Target", "Predictor", "Best Correlation", "Best Correlation Lag", "Correlations"]]

Covariance

In [ ]:
plot_series_with_predictors(target_df=hg_oi_10y_df, predictors_df=oecd_df, best_predictors_df=best_covariants, n_best=10, title="Strongest Covariant Indicators for Order Intake")
best_covariants[["Target", "Predictor", "Best Covariance", "Best Covariance Lag", "Covariances"]]

## HOMAG Order Intake Time Series Analysis

In [ ]:
#print_uniques(hg_oi_df)
hg_oi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   TIME_PERIOD  78 non-null     object 
 1   CES          78 non-null     float64
 2   CNC          78 non-null     float64
 3   CSW          78 non-null     float64
 4   Edge         78 non-null     float64
 5   N            78 non-null     float64
 6   O            78 non-null     float64
 7   Panel        78 non-null     float64
 8   T            78 non-null     float64
dtypes: float64(8), object(1)
memory usage: 5.6+ KB


In [ ]:
#print_uniques(hg_oi_10y_df)
hg_oi_10y_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   TIME_PERIOD  200 non-null    object 
 1   CES          200 non-null    float64
 2   CNC          200 non-null    float64
 3   CSW          200 non-null    float64
 4   Edge         200 non-null    float64
 5   N            200 non-null    float64
 6   O            200 non-null    float64
 7   Panel        200 non-null    float64
 8   T            200 non-null    float64
dtypes: float64(8), object(1)
memory usage: 14.2+ KB


In [ ]:
plot_time_series_with_moving_statistics(filepath="../04 Data/HOMAG/OrderIntake_series_monthly_10y.csv")

In [ ]:
plot_time_series_with_moving_statistics(filepath="../04 Data/HOMAG/OrderIntake_series_monthly.csv")

In [ ]:
hg_oi_statistics = calculate_moving_statistics(hg_oi_10y_df.index.values.tolist(), hg_oi_10y_df["T"], 4).rename(columns={"Value": "Order Intake"})
highlight_columns = ['Order Intake']
plot, plot_imgstream = create_plot(
    df=hg_oi_statistics[["Order Intake", "Moving Average", "Moving Std Dev"]].reset_index(drop=False), 
    highlight_columns=highlight_columns, 
    color_palette=color_palette,
    chart_title='Order Intake',
    xlabel='Datum',
    ylabel='Wert (in M€)',
    scale_factor=1e-6
)
plot.show()

In [ ]:
hg_oi_statistics_11_24 = calculate_moving_statistics(hg_oi_10y_df_11_24.index.values.tolist(), hg_oi_10y_df_11_24["T"], 4).rename(columns={"Value": "Order Intake"})
highlight_columns = ['Order Intake']
plot, plot_imgstream = create_plot(
    df=hg_oi_statistics_11_24[["Order Intake", "Moving Average", "Moving Std Dev"]].reset_index(drop=False), 
    highlight_columns=highlight_columns, 
    color_palette=color_palette,
    chart_title='Order Intake',
    xlabel='Datum',
    ylabel='Wert (in M€)',
    scale_factor=1e-6
)
plot.show()

Bei Gelegenheit:
- Zusätzlich zum monatlichen Verlauf der quartalsweise und jährliche AE-Verlauf mit Moving Statistics
- Box Plots AE je Monat, Quartal und Land
- Beziehungsanalysen der Feature-Zeitreihen mit dem Moving Average und der Moving Std Dev

## Best Predictors for Order Intake of OECD Data (deprecated)

In [4]:
n = 20
target_series, candidate_series_dict, best_predictors_oecd = find_best_predictors_oecd(n)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1272 entries, 1919-01-01 to 2024-12-01
Columns: 170 entries, OECD_MEI_FIN_IRLT_USA to OECD_MATERIAL_RESOURCES_MF_OECDE
dtypes: float64(170)
memory usage: 1.7 MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 78 entries, 2018-01-01 to 2024-06-01
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CES     78 non-null     float64
 1   CNC     78 non-null     float64
 2   CSW     78 non-null     float64
 3   Edge    78 non-null     float64
 4   N       78 non-null     float64
 5   O       78 non-null     float64
 6   Panel   78 non-null     float64
 7   T       78 non-null     float64
dtypes: float64(8)
memory usage: 5.5 KB
OECD_MEI_FIN_IRLT_USA
Länge: 78, Lag: 12

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.7558  , p=0.3875  , df_denom=74, df_num=1
ssr based chi2 test:   chi2=0.7864  , p=0.3752  , df=1
likelihood ratio test: chi2=0.7824  , p=

In [39]:
best_predictors_oecd[["Predictor", "Min P-Value", "Best Lag", "Comparison Period"]].head(20)
#create_p_value_df(best_predictors_oecd, n_top_predictors=n)

,Predictor,Min P-Value,Best Lag,Comparison Period
0,OECD_MEI_BTS_COS_SP_DNK,0.000180,1,"(2018-01-01 00:00:00, 2024-06-01 00:00:00)"
1,OECD_MEI_CLI_CLI_USA,0.000327,1,"(2018-01-01 00:00:00, 2024-06-01 00:00:00)"
2,OECD_MEI_BTS_COS_SP_EA19,0.000402,3,"(2018-01-01 00:00:00, 2024-06-01 00:00:00)"
3,OECD_MEI_CLI_CBC_CHE,0.000633,2,"(2018-01-01 00:00:00, 2024-06-01 00:00:00)"
4,OECD_MEI_CLI_CBC_OECD,0.001866,12,"(2018-01-01 00:00:00, 2024-06-01 00:00:00)"
5,OECD_STLABOUR_MUE_DEU,0.001979,1,"(2018-01-01 00:00:00, 2024-06-01 00:00:00)"
6,OECD_MEI_BTS_COS_SP_DEU,0.002123,3,"(2018-01-01 00:00:00, 2024-06-01 00:00:00)"
7,OECD_MEI_CLI_CBC_USA,0.002525,1,"(2018-01-01 00:00:00, 2024-06-01 00:00:00)"
8,OECD_MEI_BTS_COS_SP_AUT,0.002587,3,"(2018-01-01 00:00:00, 2024-06-01 00:00:00)"
9,OECD_MEI_FIN_IRII_DNK,0.002796,4,"(2018-01-01 00:00:00, 2024-06-01 00:00:00)"


In [ ]:
plot_series_with_predictors(target_series, best_predictors_oecd, candidate_series_dict, n_best=n)

In [ ]:
plot_series_with_predictors(target_series, best_predictors_oecd, candidate_series_dict, n_best=40, multiple_plots=True)

In [ ]:
plot_series_with_predictors(target_df=hg_oi_10y_df, predictors_df=oecd_df, best_predictors_df=best_predictors_oecd, n_best=6)

### Documentation in PowerPoint

In [ ]:
pptx_path = '../2024_06_10 DE Sales Prediction.pptx'
highlight_columns = ['Value']

plot, plot_imgstream = create_plot(
        df=hg_oi_statistics[["Value", "Moving Average", "Moving Std Dev"]].reset_index(drop=False), 
        highlight_columns=highlight_columns, 
        color_palette=color_palette,
        chart_title='Order Intake',
        xlabel='Datum',
        ylabel='Normierter Wert (in M€)',
        scale_factor=1e-6
    )

add_slide_with_image(
    pptx_path, 
    title='AE-Zeitreihe: Deskriptive Analyse', 
    img_stream=plot_imgstream
)